# Semianr 7 - Combinatoris, Graph, Boolean Logic

## 1. Solving Allocation Problem

- DFS
- Most restrictive, Least conflict

In [ ]:
import pandas as pd
import numpy as np
import time
import random
from tqdm import tqdm

In [ ]:
df_resources = pd.DataFrame([[1, "A"], [2, "B"], [3, "C"],
                             [4, "D"],[5, "E"]], columns=["ID", "Name"])

In [ ]:
df_projects = pd.DataFrame([[1, "a", "10.05.2020", "15.05.2020"],
                            [2, "b", "13.05.2020", "27.07.2020"],
                            [3, "c", "08.07.2020", "30.07.2020"],
                            [4, "d", "11.12.2020", "29.12.2020"],
                            [5, "e", "06.11.2020", "07.11.2020"]], 
                           columns=["ID", "Name", "Start Time", "End Time"])

In [ ]:
df_expertise = pd.DataFrame([[1, 1, 1], [2, 5, 3], [3, 2, 4],
                             [4, 4, 5], [5, 3, 2], [6, 2, 1],
                             [7, 3, 1], [8, 2, 2]],
                           columns=["ID", "ID_res", "ID_pro"])

### All Feasible Solutions

In [ ]:
feasible_solutions = {}

for project_index, project_row in df_projects.iterrows():
    project_id = project_row["ID"]
    res_ids = df_expertise[df_expertise['ID_pro'] == project_id]["ID_res"]
    feasible_solutions[project_id] = []
    
    for res_id in res_ids:
        feasible_solutions[project_id].append(res_id)
        
feasible_solutions

### A. Brute Force: Depth-First Search (DFS)

In [ ]:
def date_to_sec(date):
    return time.mktime(datetime.datetime.strptime(date, "%d.%m.%Y").timetuple())

In [ ]:
import time
import datetime

# keys = Project IDsdf_expertise
keys = list(feasible_solutions.keys())

def is_valid_solution(solution):
    
    for i in range(len(keys)-1):
        for j in range(i+1, len(keys)):
                
            if solution[i] == solution[j]:
                start_i = date_to_sec(df_projects["Start Time"][i])
                end_i = date_to_sec(df_projects["End Time"][i])
                
                start_j = date_to_sec(df_projects["Start Time"][j])
                end_j = date_to_sec(df_projects["End Time"][j])
                
                if start_i <= start_j:
                    if end_i >= start_j:
                        return False
                    
                elif start_i >= start_j:
                    if end_j >= start_i:
                        return False
    
    return True

# Checking a single solution
#print(is_valid_solution([1, 2, 5, 3, 4]))

#print(is_valid_solution([1, 2, 2, 3, 4]))

### First Solution

In [ ]:
answer = None

def recursive(solution=[], depth=0):    
    global answer
    #print(solution, depth)
    if(depth >= len(keys)):    
        if(is_valid_solution(solution)):
            #print(', '.join(solution))
            answer = solution
        return
    
    values = feasible_solutions[keys[depth]]
    for v in values:
        if answer:
            return answer
        next_solution = solution.copy()
        next_solution.append(str(v))
        recursive(next_solution, depth+1)
        
recursive()

### All Solutions

In [ ]:
def recursive(solution=[], depth=0):    
    #print(solution, depth)
    if(depth >= len(keys)):    
        if(is_valid_solution(solution)):
            print(', '.join(solution))
        return
    
    values = feasible_solutions[keys[depth]]
    for v in values:
        next_solution = solution.copy()
        next_solution.append(str(v))
        recursive(next_solution, depth+1)
        
recursive()

### B. Heuristic Approach

1. Most Restricted

2. Least Conflict


In [ ]:
def does_intersect(i, j):
    start_i = date_to_sec(df_projects["Start Time"][i])
    end_i = date_to_sec(df_projects['End Time'][i])
    
    start_j = date_to_sec(df_projects["Start Time"][j])
    end_j = date_to_sec(df_projects['End Time'][j])
    
    if start_i <= start_j:
        if end_i >= start_j:
            return True
        
    elif start_i >= start_j:
        if end_j >= start_i:
            return True
        
    return False

In [ ]:
df_expertise.groupby('ID_pro')["ID_res"].apply(set).to_dict()

In [ ]:
proj_res = df_expertise.groupby('ID_pro')['ID_res'].apply(set).to_dict()
keys = list(proj_res.keys())
proj_neighbours = {key: [] for key in keys}

for i in range(len(keys)):
    for j in range(i+1, len(keys)):
        if does_intersect(keys[i]-1, keys[j]-1):
            proj_neighbours[keys[i]].append(keys[j])
            proj_neighbours[keys[j]].append(keys[i])

In [ ]:
def find_smallest_nonempty():
    minlen = NRESOURCES + 1
    proj_id_sh = 0
    curr_len = 0
    
    for proj_id in proj_res:
        curr_len = len(proj_res[proj_id])
        if curr_len != 0 and curr_len < minlen:
            minlen = curr_len
            proj_id_sh = proj_id
    return proj_id_sh

In [ ]:
def count_conflicts(proj_id, res_id):
    cnt = 0
    for nbor in proj_neighbours[proj_id]:
        if nbor in proj_res and res_id in proj_res[nbor]:
            cnt + 1
    return cnt

In [ ]:
NRESOURCES = 5

solution = {}
for _ in range(len(keys)):
    proj_id = find_smallest_nonempty() # Choose the most restrictive project
    if proj_id == NRESOURCES + 1:
        print('Not complete solution')
        break
        
    # Choose resource with the least conflicts
    res_id = min(proj_res[proj_id], key=lambda x: count_conflicts(proj_id,x))
    solution[proj_id] = res_id
    proj_res[proj_id] = {}
    for nbor in proj_neighbours[proj_id]:
        if nbor in proj_res and res_id in proj_res[nbor]:
            proj_res[nbor].remove(res_id)
            
solution

## Lets use the method for large dataset

### Generate Data for Resources

In [ ]:
NRESOURCES = 1000
IDs_ = [i for i in range(1, NRESOURCES + 1)]
name_res = ["A"+str(i) for i in IDs_]
data_res = [[IDs_[i], name_res[i]] for i in range(0, NRESOURCES)]

df_resources = pd.DataFrame(data_res, columns=["ID", "Name"])
df_resources

### Generate Data for Projects

In [ ]:
NPROJECTS = 1000
IDs_ = [i for i in range(1, NPROJECTS + 1)]

mintime_ts = time.mktime(datetime.datetime.strptime("6.8.2021", "%d.%m.%Y").timetuple())
maxtime_ts = time.mktime(datetime.datetime.strptime("30.12.2021", "%d.%m.%Y").timetuple())

RANDOMTIME_start = []
RANDOMTIME_end = []

for _ in range(NPROJECTS):
    random_ts_st = random.randint(mintime_ts, maxtime_ts)
    random_ts_end = random.randint(mintime_ts, maxtime_ts)
    RANDOMTIME_start.append(datetime.datetime.fromtimestamp(min(random_ts_st, random_ts_end)).strftime("%d.%m.%Y"))
    RANDOMTIME_end.append(datetime.datetime.fromtimestamp(max(random_ts_st, random_ts_end)).strftime("%d.%m.%Y"))
    
name_proj = ["z"+str(i) for i in IDs_]
data_proj = [[IDs_[i], name_proj[i], RANDOMTIME_start[i], RANDOMTIME_end[i]] for i in range(0, NPROJECTS)]

df_projects = pd.DataFrame(data_proj, columns=["ID", "Name", "Start Time", "End Time"])
df_projects

### Generate Data for Expertise

In [ ]:
import numpy as np

NRECORDS = 20000

IDs_exp = [i for i in range(1, NRECORDS + 1)]

rand_ = list(np.random.choice(NRESOURCES, NRECORDS, replace=True))
res_ID_ = [x+1 for x in rand_]

rand = list(range(NPROJECTS)) + list(np.random.choice(NPROJECTS, NRECORDS - NPROJECTS, replace=True))
proj_ID_ = [x+1 for x in rand]
random.shuffle(proj_ID_)

data_exper = [[IDs_exp[i], res_ID_[i], proj_ID_[i]] for i in range(0, NRECORDS)]
df_expertise = pd.DataFrame(data_exper, columns=["ID", "ID_res", "ID_pro"])
df_expertise

In [ ]:
def does_intersect(i, j):
    start_i = date_to_sec(df_projects["Start Time"][i])
    end_i = date_to_sec(df_projects['End Time'][i])
    
    start_j = date_to_sec(df_projects["Start Time"][j])
    end_j = date_to_sec(df_projects['End Time'][j])
    
    if start_i <= start_j:
        if end_i >= start_j:
            return True
        
    elif start_i >= start_j:
        if end_j >= start_i:
            return True
        
    return False

def find_smallest_nonempty():
    minlen = NRESOURCES + 1
    proj_id_sh = 0
    curr_len = 0
    
    for proj_id in proj_res:
        curr_len = len(proj_res[proj_id])
        if curr_len != 0 and curr_len < minlen:
            minlen = curr_len
            proj_id_sh = proj_id
    return proj_id_sh

def count_conflicts(proj_id, res_id):
    cnt = 0
    for nbor in proj_neighbours[proj_id]:
        if nbor in proj_res and res_id in proj_res[nbor]:
            cnt + 1
    return cnt

In [ ]:
time1 = time.time()

proj_res = df_expertise.groupby('ID_pro')['ID_res'].apply(set).to_dict()
keys = list(proj_res.keys())
proj_neighbours = {key: [] for key in keys}

for i in tqdm(range(len(keys))):
    for j in range(i+1, len(keys)):
        if does_intersect(keys[i]-1, keys[j]-1):
            proj_neighbours[keys[i]].append(keys[j])
            proj_neighbours[keys[j]].append(keys[i])
    
print(f'Execution time: {time.time() - time1} sec.')

In [ ]:
solution = {}
for _ in range(len(keys)):
    proj_id = find_smallest_nonempty() # Choose the most restrictive project
    if proj_id == NRESOURCES + 1:
        print('Not complete solution')
        break
        
    # Choose resource with the least conflicts
    res_id = min(proj_res[proj_id], key=lambda x: count_conflicts(proj_id,x))
    solution[proj_id] = res_id
    proj_res[proj_id] = {}
    for nbor in proj_neighbours[proj_id]:
        if nbor in proj_res and res_id in proj_res[nbor]:
            proj_res[nbor].remove(res_id)
            
solution

## Solving Knapsak Problem

In [ ]:
class KnapsakProblem(object):
    def __init__(self, weight, value):
        self.weight = weight
        self.value = value
        self.cost = value/weight
        
    def __lt__(self, other):
        return self.cost < other.cost
        
class FractionalKnapsak(object):
    def __init__(self):
        pass
    
    def KnapsakGreedyProcc(self, W, V, M, n):
        packs = []
        for i in range(n):
            packs.append(KnapsakProblem(W[i], V[i]))
        packs.sort(reverse=True)
        
        print(packs[0].weight, packs[0].value, packs[0].cost)
        print(packs[1].weight, packs[1].value, packs[1].cost)
        print(packs[2].weight, packs[2].value, packs[2].cost)
        
        remaining = M
        result = 0
        i=0
        flag_stop = False
        while(flag_stop != True):
            if (packs[i].weight <= remaining):
                remaining -= packs[i].weight
                result += packs[i].value
                print(f'Pack {i+1}, Weight = {packs[i].weight}, Value = {packs[i].value}')
                
            else:
                i+=1
                
            if i==n:
                flag_stop = True
                
        print(f'Max value is: {result}')
        

if __name__ == '__main__':
    W = [10, 20, 30]
    V = [60, 100, 120]
    M = 50
    n = len(W)
    
    temp = FractionalKnapsak()
    temp.KnapsakGreedyProcc(W, V, M, n)

### Knapsack Problem: A Brute Force Approach

In [ ]:
def Knapsak(M, W, V, n):
    # Initial Condition
    if n==0 or M==0:
        return 0
    
    # If weight is higher than the capacity the it's not included
    if (W[n-1] > M):
        return Knapsak(M, W, V, n-1)
    
    # return eigther nth item being included or not
    else:
        return max(V[n-1]+Knapsak(M-W[n-1], W, V, n), Knapsak(M, W, V, n-1))
    
    

# Make an Example
W = [10, 20, 30]
V = [60, 100, 120]
M = 50
n = len(W)
print(Knapsak(M, W, V, n))